In [110]:
from matplotlib import cm, colors
import numpy as np
import netCDF4

cm.ScalarMappable(cmap = 'viridis').cmap(0)

(0.267004, 0.004874, 0.329415, 1.0)

In [2]:
f = netCDF4.Dataset('data/antarctica_ice_velocity_450m_v2.nc')

In [111]:
velX = f.variables['VX']
velY = f.variables['VY']
velXArray = velX[:]
velYArray = velY[:]
speed = (velXArray ** 2 + velXArray ** 2) ** 0.5
sepedMax = np.max(speed)

In [60]:
speedNorm = colors.Normalize(vmin = 0, vmax = np.max(speed))
speedScale = cm.ScalarMappable(norm = speedNorm, cmap = 'viridis').cmap

In [118]:
comp = speed.compressed()

In [ ]:
sorted(comp)